In [1]:
CONFIG_PATH = '../../config.json'
SPLIT = 'test'
IOU_THRESHOLD = 0.7

In [2]:
# Load config
import json
import numpy as np
from pathlib import Path

### Подготавливаем визуализацию

In [3]:
# Create temporary directory for predictions
pred_labels_dir = Path('runs/segment/predict/labels')
pred_images_dir = Path('runs/segment/predict')

In [4]:
with open(CONFIG_PATH, 'r') as f:
    config = json.load(f)

In [5]:
from matplotlib import pyplot as plt

## Разреженный тест

In [6]:
import pickle
with open('../../groups/gt_test_sparse.pkl', 'rb') as f:
    gt_groups = pickle.load(f)
with open('../../groups/pred_test_sparse.pkl', 'rb') as f:
    pred_groups = pickle.load(f)

In [7]:
import sys
sys.path.append('../../utils')
sys.path.append('../..')

In [8]:
from interpolate.refine_markup_by_yolo import get_box
import cv2
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm

In [9]:
def make_square_box(box, padding=10):
    x1, y1, x2, y2 = box
    width = x2 - x1
    height = y2 - y1
    
    # Make square by taking larger dimension
    size = max(width, height)
    
    # Calculate center
    center_x = (x1 + x2) / 2
    center_y = (y1 + y2) / 2
    
    # Calculate new box coordinates with padding
    new_x1 = int(center_x - size/2 - padding)
    new_y1 = int(center_y - size/2 - padding)
    new_x2 = int(center_x + size/2 + padding)
    new_y2 = int(center_y + size/2 + padding)
    
    return np.array([new_x1, new_y1, new_x2, new_y2])

In [10]:
IMAGES_DIR = Path('../../data/sparse_test/imgs')
PADDING = 0.2

In [11]:
def extract_groups(groups, output_dir):
    output_dir.mkdir(exist_ok=True)

    for group_idx, group in tqdm(enumerate(groups), total=len(groups)):
        group_dir = output_dir / str(group_idx)
        group_dir.mkdir(exist_ok=True)
        
        for obj_idx, obj in enumerate(group):
            box = get_box(obj['obj'])
            box = np.round(box).astype(int)
            p = round(max(box[2] - box[0], box[3] - box[1]) * PADDING)
            
            box = make_square_box(box, p)
            
            img_path = obj['img']
            
            img = cv2.imread(str(IMAGES_DIR / img_path.name))
            
            padded = cv2.copyMakeBorder(
                img, 
                p, p, p, p, 
                cv2.BORDER_CONSTANT, 
                value=[0,0,0]
            )
            cropped = padded[
                max(p+box[1], 0):
                max(p+box[3], 0), 
                max(p+box[0], 0):
                max(p+box[2], 0)
            ]
            if min(cropped.shape[:2]) < 5:
                print(cropped.shape, p+box[1], p+box[3], p+box[0], p+box[2])
            output_path = group_dir / f"{obj_idx}.png"
            cv2.imwrite(str(output_path), cropped)

In [12]:
output_dir = Path('../../data/sparse_test/gt_groups')
extract_groups(gt_groups, output_dir)

100%|██████████| 792/792 [04:45<00:00,  2.77it/s]


In [13]:
output_dir = Path('../../data/sparse_test/pred_groups')
extract_groups(pred_groups, output_dir)

100%|██████████| 879/879 [04:04<00:00,  3.60it/s]
